In [1]:
import json
from urllib.request import urlopen
import time

import pandas as pd

In [ ]:
baseurl = 'https://api.pushshift.io/reddit/search/submission/?subreddit=india&sort=desc&sort_type=created_utc&'

period = 1 # in years

week_seconds = 7 * 24 * 60 * 60
week_count = 1
start_seconds = int(time.time()) - ((week_count - 1) * week_seconds)

all_data = []

while week_count <= int(period * 52):
    before, after = start_seconds, start_seconds - week_seconds
    url = baseurl + 'after={}&before={}&size=1000'.format(after, before)
    start_seconds = after
    r = urlopen(url).read()
    data = json.loads(r.decode('utf-8'))
    all_data += data['data']
    print(f'Week {week_count}/{period*52} done')
    week_count += 1
    
    
all_data_DF = pd.DataFrame(all_data)
all_data_DF.to_csv('reddit_data.csv')

In [2]:
keep_features = ['author', 'title', 'selftext', 'created_utc', 'domain', 'score', 'link_flair_text', 'num_comments']

In [3]:
collected_data = pd.read_csv('reddit_data.csv')
collected_data = collected_data[keep_features]
collected_data

/home/sushmitha/Desktop/AI/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,4,13,14,15,16,18,19,21,23,26,29,43,47,49,51,52,53,56,57,60,61,62,63,69,79,83,92,93,94,95,96,97) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,author,title,selftext,created_utc,domain,score,link_flair_text,num_comments
0,ParmanuMan,Bhilwara(Rajasthan) model of ruthless containm...,NaN,1586247215,moneycontrol.com,1,NaN,0
1,ayyushsoolanki,07/04/2020,NaN,1586246882,imgur.com,1,All CAPS.,0
2,dlisfyn,Nothing Wrong With People Bursting Firecracker...,NaN,1586246760,ndtv.com,1,Coronavirus,1
3,ayyushsoolanki,Happy World Health Day amid times when world i...,NaN,1586246719,i.redd.it,1,NaN,0
4,Explria,Aurangabad doctor booked for social media post...,NaN,1586246673,indianexpress.com,1,Politics,0
5,ayyushsoolanki,Happy World Health Day amid times when world i...,[removed],1586246406,self.india,1,Non-Political,0
6,ayyushsoolanki,Happy World Health Day amid times when world i...,[removed],1586246392,self.india,1,Non-Political,0
7,pkmcreation,How to Get Healthy Life: अच्छा स्वास्थ्य कैसे ...,NaN,1586246022,pkmcreation.com,1,Food,0
8,wordswithmagic,Kerala high court allows cat owner to buy pet ...,NaN,1586246020,m.timesofindia.com,1,Coronavirus,0
9,ayyushsoolanki,Happy Health Day amid the times when world is ...,NaN,1586245936,i.redd.it,1,Non-Political,0


In [4]:
with_flair = collected_data[collected_data.link_flair_text.notnull()]
with_flair.reset_index(drop=True, inplace=True)
with_flair

,author,title,selftext,created_utc,domain,score,link_flair_text,num_comments
0,ayyushsoolanki,07/04/2020,NaN,1586246882,imgur.com,1,All CAPS.,0
1,dlisfyn,Nothing Wrong With People Bursting Firecracker...,NaN,1586246760,ndtv.com,1,Coronavirus,1
2,Explria,Aurangabad doctor booked for social media post...,NaN,1586246673,indianexpress.com,1,Politics,0
3,ayyushsoolanki,Happy World Health Day amid times when world i...,[removed],1586246406,self.india,1,Non-Political,0
4,ayyushsoolanki,Happy World Health Day amid times when world i...,[removed],1586246392,self.india,1,Non-Political,0
5,pkmcreation,How to Get Healthy Life: अच्छा स्वास्थ्य कैसे ...,NaN,1586246022,pkmcreation.com,1,Food,0
6,wordswithmagic,Kerala high court allows cat owner to buy pet ...,NaN,1586246020,m.timesofindia.com,1,Coronavirus,0
7,ayyushsoolanki,Happy Health Day amid the times when world is ...,NaN,1586245936,i.redd.it,1,Non-Political,0
8,ashish_59,Feeling bored at home during COVID-19 lockdown?,[removed],1586245533,self.india,1,Business/Finance,0
9,sbraja1989,I request you all to maintain self quarantine....,NaN,1586245495,i.redd.it,1,Non-Political,0


In [5]:
from collections import Counter
counts = Counter(with_flair['link_flair_text'])
counts

Counter({'All CAPS.': 164,
         'Coronavirus': 1593,
         'Politics': 7781,
         'Non-Political': 6880,
         'Food': 368,
         'Business/Finance': 1866,
         'Science/Technology': 1432,
         'AskIndia': 4468,
         'Photography': 935,
         'Policy/Economy': 1210,
         'Sports': 569,
         'Low-effort self-post.': 108,
         'CAA-NRC-NPR': 21,
         '| Low-effort Self Post |': 2,
         '| Not specific to India |': 5,
         '| Unverified Content / Disreputed Source |': 3,
         '| Repost |': 2,
         'Scheduled': 146,
         '| [OLD] Content |': 1,
         'Meta.': 1,
         'Megathread': 1,
         'Shitpost': 2,
         '| Image Rule Violation |': 2,
         '8041227c-6517-11ea-b83e-0e7048fc0c5b': 1,
         'Demonetization': 163,
         'CAA-NRC': 403,
         '77f04f12-7ea0-11e3-ac66-22000a0b8292': 6,
         'Low Quality/Non OC Meme': 1,
         'Personal/Unverified Twitter.': 2,
         '[R]eddiquette': 699,